import your train dataset ....                
folder structure                   
 data/        
 |-positive ( your images)           
 |-anchor ( your anchoe images - you....)          
 |-negative ( LFW images or any other images than you )

In [ ]:
import shutil

zip_path = "data.zip"
extract_to = "data"

shutil.unpack_archive(zip_path, extract_to)
print("Unzipped successfully")


Unzipped successfully


In [ ]:
import os
import cv2
import numpy as np

preprocess the image to fit our embedding layer


In [ ]:
def preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (100,100))
    img = tf.cast(img, tf.float32) / 255.0
    return img


the embedding layer where we transform the image to a numerical value (tf tensor)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D,
    GlobalAveragePooling2D, Dense, Input
)
from tensorflow.keras.models import Model

class L2Normalize(tf.keras.layers.Layer):
    def call(self, x):
        return tf.math.l2_normalize(x, axis=1)

def make_embedding():
    inp = Input(shape=(100,100,3), name="input_image")

    # Convolutional base
    x = Conv2D(32, (7,7), activation='relu', padding='same')(inp)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(64, (5,5), activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)

    # Project to embedding space
    x = Dense(128, activation='relu')(x)
    x = L2Normalize()(x)

    return Model(inputs=inp, outputs=x, name="embedding")


function to find distance between the images ( eucilidian distance )

In [ ]:
class L2Distance(tf.keras.layers.Layer):
    def call(self, inputs):
        e1, e2 = inputs
        return tf.norm(e1 - e2, axis=1, keepdims=True)


In [ ]:
def make_siamese_model():
    embedding = make_embedding()

    img1 = Input(shape=(100,100,3), name="img1")
    img2 = Input(shape=(100,100,3), name="img2")

    e1 = embedding(img1)
    e2 = embedding(img2)

    dist = L2Distance()([e1, e2])

    return Model([img1, img2], dist, name="siamese")

siamese_model=make_siamese_model()

loss function
   loss is less for same image and high for different img
   

In [ ]:
def contrastive_loss(y_true, d, margin=1.0):
    y_true = tf.cast(y_true, tf.float32)
    pos_loss = y_true * tf.square(d)
    neg_loss = (1 - y_true) * tf.square(tf.maximum(margin - d, 0))
    return tf.reduce_mean(pos_loss + neg_loss)


train and weight update function


In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(batch):
    img1, img2, labels = batch

    with tf.GradientTape() as tape:
        d = siamese_model([img1, img2])
        loss = contrastive_loss(labels, d)

    grads = tape.gradient(loss, siamese_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, siamese_model.trainable_variables))

    return loss



train dataset and test dataset creation

In [ ]:
pos_path=os.path.join('data',"data",'positive')

neg_path=os.path.join('data',"data",'negative')
anc_path=os.path.join('data',"data",'anchor')



In [ ]:
anchor_imgs=tf.data.Dataset.list_files(anc_path+'/*.jpg')
positive_imgs=tf.data.Dataset.list_files(pos_path+'/*.jpg')
negative_imgs=tf.data.Dataset.list_files(neg_path+'/*.jpg')

In [ ]:
def preprocessing_twin(anch_file,test_file,num):
    return (preprocess_image(anch_file),preprocess_image(test_file),num)

In [ ]:
anchor=anchor_imgs.take(100)
positive=positive_imgs.take(100)
negative=negative_imgs.take(100)

In [ ]:
positives=tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives=tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
print(len(negatives))
data=positives.concatenate(negatives)
len(data)

100


200

In [ ]:
data=data.map(preprocessing_twin)
data=data.cache()
data=data.shuffle(buffer_size=1024)

In [ ]:
train_data=data.take(round(len(data)*.7))
print(len(train_data))
train_data=train_data.batch(16)
train_data=train_data.prefetch(8)

140


In [ ]:
test_data=data.skip(round(len(data)*.7))
test_data=data.take(round(len(data)*.3))
print(len(test_data))
test_data=test_data.batch(16)
test_data=test_data.prefetch(8)

60


model training

In [ ]:
EPOCHS = 10
STEPS_PER_EPOCH = 100

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    for step, batch in enumerate(train_data):

        loss = train_step(batch)

        if step % 20 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}")



Epoch 1/10
Step 0, Loss: 0.5669

Epoch 2/10
Step 0, Loss: 0.3813

Epoch 3/10
Step 0, Loss: 0.2618

Epoch 4/10
Step 0, Loss: 0.2884

Epoch 5/10
Step 0, Loss: 0.2453

Epoch 6/10
Step 0, Loss: 0.2878

Epoch 7/10
Step 0, Loss: 0.2895

Epoch 8/10
Step 0, Loss: 0.2935

Epoch 9/10
Step 0, Loss: 0.2827

Epoch 10/10
Step 0, Loss: 0.2672


testing phase


as i dont have a proper web cam i am upload a floder with refernce images ( me ) and a test input folder ( photos of me and others ) if u dont have a proper web cam i suggest you do this

In [ ]:
import os

def load_reference_embeddings(embedding_model, ref_folder):
    ref_embeddings = []

    for file in os.listdir(ref_folder):
        path = os.path.join(ref_folder, file)
        emb = embedding_model(preprocess_image_verify(path), training=False)
        ref_embeddings.append(emb)

    return ref_embeddings


verification function

In [ ]:
import numpy as np

def verify_image(embedding_model, ref_embeddings, img_path, threshold=0.6):
    test_emb = embedding_model(preprocess_image(img_path), training=False)

    distances = [
        tf.norm(test_emb - ref_emb, axis=1).numpy()[0]
        for ref_emb in ref_embeddings
    ]

    mean_distance = np.mean(distances)

    return mean_distance, mean_distance < threshold


In [ ]:
import traceback

def verify_folder(embedding_model, ref_embeddings, folder, threshold=0.6):
    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        print(f"\nProcessing: {file}")

        try:
            img = preprocess_image_verify(path)
            print("  preprocess OK, shape:", img.shape)

            test_emb = embedding_model(img, training=False)
            print("  embedding OK, shape:", test_emb.shape)

            distances = []
            for ref_emb in ref_embeddings:
                d = tf.norm(test_emb - ref_emb, axis=1)
                distances.append(float(d.numpy()[0]))

            mean_distance = sum(distances) / len(distances)
            print("  mean distance:", mean_distance)

            print("  RESULT:", "YOU" if mean_distance < threshold else "NOT YOU")

        except Exception as e:
            print("  ❌ ERROR:")
            traceback.print_exc()


In [ ]:
def preprocess_image_verify(path):
    img = preprocess_image(path)      # reuse training logic
    return tf.expand_dims(img, axis=0)


fucntion calling and seeing if our model works properly

reference -> your images         
input_images -> test images

In [ ]:
embedding_model = make_embedding()  # or load it
ref_embeddings = load_reference_embeddings(
    embedding_model,
    "reference/reference"
)

results = verify_folder(
    embedding_model,
    ref_embeddings,
    "/content/input_images/input_images",
    threshold=0.05
)




Processing: licensed-image.jpg
  preprocess OK, shape: (1, 100, 100, 3)
  embedding OK, shape: (1, 128)
  mean distance: 0.10020907750974099
  RESULT: NOT YOU

Processing: Legends-Profile_Eric-Cantona1523462164078.jpg
  preprocess OK, shape: (1, 100, 100, 3)
  embedding OK, shape: (1, 128)
  mean distance: 0.07425883179530501
  RESULT: NOT YOU

Processing: input_image.jpg
  preprocess OK, shape: (1, 100, 100, 3)
  embedding OK, shape: (1, 128)
  mean distance: 0.15967688585321108
  RESULT: NOT YOU

Processing: gettyimages-2218379589-612x612.jpg
  preprocess OK, shape: (1, 100, 100, 3)
  embedding OK, shape: (1, 128)
  mean distance: 0.1989547312259674
  RESULT: NOT YOU

Processing: 20260116_090914_030.jpg
  preprocess OK, shape: (1, 100, 100, 3)
  embedding OK, shape: (1, 128)
  mean distance: 0.04400862823240459
  RESULT: YOU

Processing: 20260116_090914_007.jpg
  preprocess OK, shape: (1, 100, 100, 3)
  embedding OK, shape: (1, 128)
  mean distance: 0.03687350001807014
  RESULT: YOU

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
# Save weights
siamese_model.save('siamesemodel_with_mean_distance.keras')